<a href="https://colab.research.google.com/github/Chinmaysahoo03/Multimodal_GenAI_Assistant/blob/main/multimodal_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain-groq transformers torch diffusers accelerate pillow moviepy opencv-python

import torch
from diffusers import StableDiffusionPipeline, TextToVideoSDPipeline
from PIL import Image
from IPython.display import display, Video
import warnings
warnings.filterwarnings("ignore")  # Suppress minor warnings for cleaner output

# Setup Groq + LangChain for Chatbot
from google.colab import userdata
groq_api = userdata.get('groq_api')  # Add your Groq key in Colab Secrets
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Initialize free Groq model (Gemma-2-9B-IT)
llm = ChatGroq(api_key=groq_api, model="gemma2-9b-it", temperature=0.7)

# Prompt template for conversational style with history
prompt_template = PromptTemplate(
    input_variables=["history", "input"],
    template="{history}\nHuman: {input}\nAssistant:"
)

chain = LLMChain(llm=llm, prompt=prompt_template)

# Setup Text-to-Image: Stable Diffusion
pipe_image = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
if torch.cuda.is_available():
    pipe_image = pipe_image.to("cuda")
    print("GPU enabled for faster image generation!")

# Setup Text-to-Video: ModelScope Text-to-Video
pipe_video = TextToVideoSDPipeline.from_pretrained(
    "damo-vilab/text-to-video-ms-1.7b",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    variant="fp16"
)
if torch.cuda.is_available():
    pipe_video = pipe_video.to("cuda")
    print("GPU enabled for faster video generation!")
else:
    print("Running on CPU for video generation.")

print("Models loaded successfully! Groq for chat, Stable Diffusion for images, ModelScope for videos.")

def generate_chat_response(user_input, history=""):
    """
    Generate a text response using Groq and LangChain.

    Args:
        user_input (str): User's input query.
        history (str): Conversation history.

    Returns:
        tuple: (response text, updated history)
    """
    input_dict = {"input": user_input, "history": history}
    response = chain.invoke(input_dict)
    response_text = response['text'] if 'text' in response else response.get('output', 'No valid response')
    new_history = f"{history}\nHuman: {user_input}\nAssistant: {response_text}"
    return response_text.strip(), new_history

def generate_image(prompt, negative_prompt="blurry, low quality", num_steps=50):
    """
    Generate an image from a text prompt using Stable Diffusion.

    Args:
        prompt (str): Description of the image to generate.
        negative_prompt (str): What to avoid in the image (default: "blurry, low quality").
        num_steps (int): Number of inference steps (default: 50).

    Returns:
        Image: Generated PIL image.
    """
    image = pipe_image(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=num_steps
    ).images[0]

    display(image)  # Shows inline in Colab
    image.save("generated_image.png")
    print(f"Image saved as generated_image.png")
    return image

def generate_video(prompt, num_frames=16, num_steps=25, fps=8):
    """
    Generate a video from a text prompt using ModelScope Text-to-Video pipeline.

    Args:
        prompt (str): Description of the video to generate.
        num_frames (int): Number of frames in the video (default: 16).
        num_steps (int): Number of inference steps for generation (default: 25).
        fps (int): Frames per second for the output video (default: 8).

    Returns:
        str: Path to the saved video file.
    """
    video_frames = pipe_video(
        prompt,
        num_inference_steps=num_steps,
        height=320,  # Adjust resolution as needed
        width=512,
        num_frames=num_frames
    ).frames[0]  # Access the frames array

    from diffusers.utils import export_to_video
    video_path = "generated_video.mp4"
    export_to_video(video_frames, video_path, fps=fps)

    display(Video(video_path, embed=True))
    print(f"Video saved as {video_path}")
    return video_path

# Test standalone functionalities
print("\n--- Testing Standalone Chat ---")
test_input = "Hello, tell me about generative AI."
response, history = generate_chat_response(test_input)
print(f"User: {test_input}")
print(f"Bot: {response}")

print("\n--- Testing Standalone Image Generation ---")
test_image_prompt = "a friendly robot chatting with a human in a futuristic room"
generate_image(test_image_prompt)

print("\n--- Testing Standalone Video Generation ---")
test_video_prompt = "a cat playing with a ball in a park"
generate_video(test_video_prompt)

# Interactive multimodal assistant loop
print("\nMultimodal AI Assistant Ready! Type 'quit' to exit. Use 'generate image: [description]' for images or 'generate video: [description]' for videos.")
history = ""  # Stateful history for Groq

while True:
    user_input = input("\nYou: ")
    if user_input.lower() == 'quit':
        break

    # Check for image generation command
    if user_input.lower().startswith('generate image:'):
        image_prompt = user_input[15:].strip()  # Extract prompt after "generate image:"
        print("Generating image...")
        generate_image(image_prompt)
        continue

    # Check for video generation command
    if user_input.lower().startswith('generate video:'):
        video_prompt = user_input[15:].strip()  # Extract prompt after "generate video:"
        print("Generating video...")
        generate_video(video_prompt)
        continue

    # Generate text response with Groq
    response, history = generate_chat_response(user_input, history)
    print(f"Assistant: {response}")

    # Optional: Offer to generate image or video based on response
    gen_choice = input("Generate an image (i) or video (v) based on this response? (i/v/n): ")
    if gen_choice.lower() == 'i':
        image_prompt = response[:100] + "..."  # Use response as prompt (truncate if long)
        print("Generating contextual image...")
        generate_image(image_prompt)
    elif gen_choice.lower() == 'v':
        video_prompt = response[:100] + "..."  # Use response as prompt (truncate if long)
        print("Generating contextual video...")
        generate_video(video_prompt)

# Run a scripted demo to showcase integration
demo_inputs = [
    "Hi, what's generative AI?",
    "Can you describe a scene of AI creating art?",
    "generate image: AI painting a masterpiece",
    "generate video: AI robot dancing"
]

history = ""
for user_input in demo_inputs:
    if user_input.lower().startswith('generate image:'):
        image_prompt = user_input[15:].strip()
        print(f"\n--- Image Request: {image_prompt} ---")
        generate_image(image_prompt)
    elif user_input.lower().startswith('generate video:'):
        video_prompt = user_input[15].strip()
        print(f"\n--- Video Request: {video_prompt} ---")
        generate_video(video_prompt)
    else:
        print(f"\nYou: {user_input}")
        response, history = generate_chat_response(user_input, history)
        print(f"Assistant: {response}")
        print("---")